Source: https://github.com/eliasmelul/finding_schitts/blob/master/Recommender_System_FMSC.ipynb

In [6]:
%pip install pandas
%pip install scikit-learn
%pip install matplotlib

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\vishr\VSCodeProjects\CS 222\Project\api\Scripts\python.exe -m pip install --upgrade pip' command.


Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\vishr\VSCodeProjects\CS 222\Project\api\Scripts\python.exe -m pip install --upgrade pip' command.


     ---------------------------------------- 7.6/7.6 MB 2.4 MB/s eta 0:00:00
  Using cached pyparsing-3.0.9-py3-none-any.whl (98 kB)
     ---------------------------------------- 55.4/55.4 KB 2.8 MB/s eta 0:00:00
  Using cached cycler-0.11.0-py3-none-any.whl (6.4 kB)
     -------------------------------------- 160.2/160.2 KB 3.3 MB/s eta 0:00:00
     ---------------------------------------- 2.5/2.5 MB 22.8 MB/s eta 0:00:00
     ---------------------------------------- 1.0/1.0 MB 32.1 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\vishr\VSCodeProjects\CS 222\Project\api\Scripts\python.exe -m pip install --upgrade pip' command.


In [7]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

%matplotlib inline

In [11]:
norm_df = pd.read_csv('https://s3.us-east-2.amazonaws.com/www.findingmyschittscreek.com/Data/normalized_df_sub.csv', index_col=0)

In [12]:
def from_city_cosSim(data, name):

    try:
        Xs = data[data.City == name].drop('City',1)
        Col_A = data[data.City != name].City
        Ys = data[data.City != name].drop('City',1)

        cosSim = cosine_similarity(X=Xs,Y=Ys)
        sim = list(cosSim[0])
        cty = list(Col_A)
        comb = {"City":cty,"Similarity":sim}
        dfdf = pd.DataFrame(comb).reset_index()

        # Adding a row with the Y City
        currCity = {"City":name,"Similarity":1}
        curr = pd.DataFrame(currCity, index=[0])

        # Concatenate to finalize DF
        dfdf = pd.concat([dfdf,curr], sort=False).reset_index(drop=True).drop('index',1)
        return(dfdf)
    except:
        print("Wrong input: this entry will be ignored")

In [13]:


def input_cities(numShow=10):
    # create class that defines cities
    class rated_city:
        def __init__(self, city):
            self.city = city
    
    #Loop to input cities based on the user
    add_city = True
    userInput = []
    w = 0
    while add_city == True:
        city_name = input("City (Include state - Ex. New York, NY): ")
        userInput.append(city_name)
        simSim = from_city_cosSim(data=norm_df, name=city_name)
        try:
            cosSim = cosSim.merge(simSim, how='inner', on='City')
        except:
            cosSim = simSim
        
        city = rated_city(city_name)
        cont = input("Do you want to include another city?")
        add_city = cont.lower() in ['yes','true','of course','y','si','1']
        w+=1
    
    simCols = cosSim.drop("City",1)
    cits = cosSim.City
        
    for i, row in simCols.iterrows():
        simCols.at[i,'SumVal'] = row.sum()/w
    simi = simCols.SumVal
    
    out = {"City":cits,"Score":simi}
    out = pd.DataFrame(out).set_index("City").drop(userInput)
    out = out.sort_values('Score', ascending=False)
    
    return out.head(numShow)      



In [14]:

input_cities()

C:\Users\vishr\AppData\Local\Temp\ipykernel_64924\2510333559.py:4: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  Xs = data[data.City == name].drop('City',1)
C:\Users\vishr\AppData\Local\Temp\ipykernel_64924\2510333559.py:6: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  Ys = data[data.City != name].drop('City',1)
C:\Users\vishr\AppData\Local\Temp\ipykernel_64924\2510333559.py:19: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  dfdf = pd.concat([dfdf,curr], sort=False).reset_index(drop=True).drop('index',1)
C:\Users\vishr\AppData\Local\Temp\ipykernel_64924\2353394123.py:25: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  simCols = cosSim.drop("Ci

,Score
City,
"Chicago, IL",0.799584
"Seattle, WA",0.760618
"Washington, DC",0.760189
"Philadelphia, PA",0.747166
"Pittsburgh, PA",0.746654
"Minneapolis, MN",0.743887
"Modesto, CA",0.726210
"Baltimore, MD",0.724874
"Tampa, FL",0.720664
